In [ ]:
import pandas as pd
import os

# Parâmetros Estabelecimentos
dtypes_EMPRESA = {'CNPJ_BASE': 'category'}

diretorio_empresa = r'base_csv_empresas/'
all_files_estabelecimentos = list(filter(lambda x: '.csv' in x, os.listdir(diretorio_empresa)))

for file_empresa in all_files_estabelecimentos:
    d_empresa = pd.read_csv(f'{diretorio_empresa}{file_empresa}',dtype=dtypes_EMPRESA,error_bad_lines=False,sep=';', nrows=10)
    print(d_empresa)

In [2]:
# libs
import pandas as pd
import json
import re
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")

dados = pd.read_csv('DB_SHOW_APPLE.csv')
dados.head(2)

,CNPJ,RAZAO_SOCIAL,NOME_FANTASIA,RUA,NUMERO,COMPLEMENTO,BAIRRO,CIDADE,ESTADO,CEP,TELEFONE,EMAIL,CNAE_PRINCIPAL,CNAE_DESCRICAO
0,58541000170,Q COISA BOA BAR LTDA,BUTECO TOMAUMA,RUA MATEUS LEME,2631,NaN,BOM RETIRO,CURITIBA,PR,80520174.0,(41)98713461,DELIMA.ADV@GMAIL.COM,5611205,Restaurantes e similares
1,108202000151,ARMAZEM COQUEIRO LTDA,ARMAZEM COQUEIRO,ESTRADA DO CABRAL,0,NaN,CABRAL,PARATY,RJ,23970000.0,(24)337127,VITRINA@VITRINACONTABILIDADE.COM.BR,5611205,Restaurantes e similares


In [9]:
dados['COORDENADAS'] = dados['CEP'].apply(lambda x: geolocator.geocode((x), timeout=10))
#dados['COORDENADAS'] = dados[''].apply(lambda x: geolocator.geocode(x, timeout=10))
dados['CEP'].head(10)

0    80520174.0
1    23970000.0
2    88865000.0
3    95765000.0
4    29907040.0
5    15880000.0
6    13960000.0
7    13650000.0
8    72225215.0
9    65605515.0
Name: CEP, dtype: float64

In [11]:

dados['LATITUDE'] = dados.apply(lambda row: row['COORDENADAS'].latitude if row['COORDENADAS'] is not None else None, axis=1)
dados['LONGITUDE']  = dados.apply(lambda row: row['COORDENADAS'].longitude if row['COORDENADAS'] is not None else None, axis=1)


In [15]:
dados = dados.sort_values(by='LATITUDE', ascending=False)


In [29]:
import random
aleatorio = random.randint(0, 3)
horario = ["""domingo 
Fechado

segunda-feira
11:00–15:00
18:00–23:00

terça-feira
11:00–15:00
18:00–23:00

quarta-feira
11:00–15:00
18:00–23:00

quinta-feira
11:00–15:00
18:00–23:00

sexta-feira
11:00–15:00
18:00–23:00

sábado
11:00–15:00
18:00–23:00""",
"""domingo
Fechado

segunda-feira
11:00–15:00

terça-feira
11:00–15:00

quarta-feira
11:00–15:00

quinta-feira
11:00–15:00

sexta-feira
11:00–15:00

sábado
Fechado""",
"""
domingo
Fechado

segunda-feira
10:00–14:30
18:00–00:00

terça-feira
10:00–14:30
18:00–00:00

quarta-feira
10:00–14:30
18:00–00:00

quinta-feira
10:00–14:30
18:00–00:00

sexta-feira
10:00–14:30
18:00–00:00

sábado
10:00–14:30
18:00–00:00""",
"""domingo
Fechado

segunda-feira
11:00–15:30

terça-feira
11:00–15:30

quarta-feira
11:00–15:30

quinta-feira
11:00–15:30

sexta-feira
11:00–15:30

sábado
11:00–15:00"""]
for horario in dados['CNPJ']:
    dados['HORARIO_FUNCIONAMENTO'] = horario[aleatorio]
dados



,CNPJ,RAZAO_SOCIAL,NOME_FANTASIA,RUA,NUMERO,COMPLEMENTO,BAIRRO,CIDADE,ESTADO,CEP,TELEFONE,EMAIL,CNAE_PRINCIPAL,CNAE_DESCRICAO,COORDENADAS,LATITUDE,LONGITUDE,HORARIO_FUNCIONAMENTO
286,5368000142,ROSE & DETE BAR E MERCEARIA LTDA,Indisponível,AVENIDA DR VITAL BRASIL,1500,BLOCO F CASA 4,BUTANTA,SAO PAULO,SP,5503000.0,(nan)nan,NaN,5611203,Bares e outros estabelecimentos especializados...,(г. Нижняя Салда — п. Басьяновский — с. Медвед...,58.153720,60.751312,domingo\nFechado\n\nsegunda-feira\n11:00–15:30...
365,11710000117,PRODUTOS ALIMENTICIOS MACROZEN LTDA,Indisponível,RUA CONSELHEIRO FURTADO,702,NaN,LIBERDADE,SAO PAULO,SP,1511000.0,(nan)nan,NaN,5611201,Bares e outros estabelecimentos especializados...,"(65К-1511000, Невьянский городской округ, Свер...",57.086297,60.167544,domingo\nFechado\n\nsegunda-feira\n11:00–15:30...
380,13137000180,A.M. ALIMENTACAO COMERCIAL EIRELI,Indisponível,AVENIDA M COELHO DE AGUIAR,215,BLOCO B LOJA 86 C,JD SAO LUIS,SAO PAULO,SP,5805000.0,(nan)nan,FISCAL@CONTABILMODELAR.COM.BR,5611201,Bares e outros estabelecimentos especializados...,"(улица Томилина, Билимбай, городской округ Пер...",56.963808,59.820205,domingo\nFechado\n\nsegunda-feira\n11:00–15:30...
357,9711000127,ESTELA IMACULADA CONCEICAO ZANOLA DE SOUZA,Indisponível,RUA AUGUSTA,2231,NaN,CERQUEIRA CESAR,SAO PAULO,SP,1413000.0,(nan)nan,NaN,5611201,Bares e outros estabelecimentos especializados...,"(65К-1413000, Чатлык, Красноуфимский округ, Св...",56.805117,57.695403,domingo\nFechado\n\nsegunda-feira\n11:00–15:30...
106,5234000121,BAR E LANCHES PINGO AZUL,Indisponível,ALAMEDA JOAQUIM EUGENIO DE LIMA,212,NaN,PARAISO,SAO PAULO,SP,1403000.0,(nan)nan,NaN,5611204,Lanchonetes casas de chá de sucos e similares,(г. Красноуфимск — с. Нижнеиргинское — с. Крас...,56.664307,57.717982,domingo\nFechado\n\nsegunda-feira\n11:00–15:30...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,249743000108,DIGESSO & EDMILSON LTDA,AQUARIUS BAR,AVENIDA JEAN EMILE FAVRE,1574,B,VILA DO IPSEP,RECIFE,PE,51190450.0,(nan)nan,NaN,5612100,Serviços ambulantes de alimentação,None,NaN,NaN,domingo\nFechado\n\nsegunda-feira\n11:00–15:30...
491,262687000133,MONICA MARIA ALVES DA SILVA,NUTRIDIET NUTRICAO E DIETOTERAPIA,RUA SENADOR J HENRIQUE,103,LOJA 04 TERREO,ILHA DO LEITE,RECIFE,PE,50070460.0,(nan)nan,NaN,5612100,Serviços ambulantes de alimentação,None,NaN,NaN,domingo\nFechado\n\nsegunda-feira\n11:00–15:30...
494,265535000194,PETISQUEIRA TIA ANGELA LTDA,BARRACA DA TIA ANGELA,AVENIDA DR JOSE PEIXE ABBADE,4128,QUIOSQUE,SUARAO,ITANHAEM,SP,11740000.0,(nan)nan,NaN,5612100,Serviços ambulantes de alimentação,None,NaN,NaN,domingo\nFechado\n\nsegunda-feira\n11:00–15:30...
495,270352000167,JUAN MARIA HERNANDEZ FRANCIA,URUGUAY LANCHES,RUA JOAO ALVES DELFINO,SN,PRACA BOM J C VERDE,CENTRO,RIBEIRAO BONITO,SP,13580000.0,(nan)nan,NaN,5612100,Serviços ambulantes de alimentação,None,NaN,NaN,domingo\nFechado\n\nsegunda-feira\n11:00–15:30...


In [31]:
dados = dados[['CNPJ', 'RAZAO_SOCIAL', 'NOME_FANTASIA', 'RUA', 'NUMERO', 'COMPLEMENTO',
       'BAIRRO', 'CIDADE', 'ESTADO', 'CEP', 'TELEFONE', 'EMAIL',
       'CNAE_PRINCIPAL', 'CNAE_DESCRICAO', 'LATITUDE',
       'LONGITUDE', 'HORARIO_FUNCIONAMENTO']]

In [32]:
dados.to_json("db_SIGA_05_APPLE.json", orient="records")
dados = dados[['CNPJ', 'RAZAO_SOCIAL', 'NOME_FANTASIA', 'RUA', 'NUMERO', 'COMPLEMENTO',
       'BAIRRO', 'CIDADE', 'ESTADO', 'CEP','LATITUDE',
       'LONGITUDE','TELEFONE', 'EMAIL','CNAE_PRINCIPAL', 'CNAE_DESCRICAO']]
dados.to_json("db_SIGA_04_APPLE.json", orient="records")
dados = dados[['CNPJ', 'RAZAO_SOCIAL', 'NOME_FANTASIA', 'RUA', 'NUMERO', 'COMPLEMENTO',
       'BAIRRO', 'CIDADE', 'ESTADO', 'CEP','LATITUDE',
       'LONGITUDE','TELEFONE', 'EMAIL']]
dados.to_json("db_SIGA_03_APPLE.json", orient="records")
dados = dados[['CNPJ', 'RAZAO_SOCIAL', 'NOME_FANTASIA', 'RUA', 'NUMERO', 'COMPLEMENTO',
       'BAIRRO', 'CIDADE', 'ESTADO', 'CEP','LATITUDE',
       'LONGITUDE']]
dados.to_json("db_PUBLIC_BASE_02_APPLE.json", orient="records")
dados = dados[['CNPJ', 'RAZAO_SOCIAL', 'NOME_FANTASIA', 'RUA', 'NUMERO', 'COMPLEMENTO',
       'BAIRRO', 'CIDADE', 'ESTADO', 'CEP']]
dados.to_json("db_PUBLIC_BASE_01_APPLE.json", orient="records")



